# Layers and Blocks

To begin, we revisit the code
that we used to implement MLPs

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.1619, -0.0312,  0.0278, -0.4482, -0.2112, -0.0350,  0.1517,  0.2062,
          0.1594,  0.0302],
        [-0.1790,  0.0224, -0.0154, -0.5712, -0.1834,  0.1255,  0.1926,  0.1549,
          0.2008,  0.0573]], grad_fn=<AddmmBackward>)

`nn.Sequential` defines a special kind of `Module`

A Custom Block

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

Instantiate the MLP's layers
and subsequently invoke these layers

In [3]:
net = MLP()
net(X)

tensor([[-2.0535e-02, -1.4996e-01, -1.7301e-02,  4.6092e-02, -1.2587e-01,
          1.0544e-01,  2.0127e-01,  6.6108e-02,  7.7090e-02,  4.0703e-05],
        [ 1.2835e-01, -1.0584e-01, -1.5432e-02, -2.5493e-02, -2.1751e-01,
          1.7708e-01,  2.8758e-01,  1.8627e-01,  1.2677e-01, -4.9928e-02]],
       grad_fn=<AddmmBackward>)

The Sequential Block

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1731, -0.0926, -0.0158,  0.0505, -0.0422,  0.0173,  0.3812,  0.0247,
         -0.0344,  0.1177],
        [ 0.3422, -0.1667, -0.2005, -0.1068, -0.1901, -0.0434,  0.3102,  0.0271,
         -0.0195,  0.2211]], grad_fn=<AddmmBackward>)

Executing Code in the Forward Propagation Function

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(-0.2325, grad_fn=<SumBackward0>)

Mix and match various
ways of assembling blocks together

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.1140, grad_fn=<SumBackward0>)